# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [13]:
import pandas as pd
import numpy as np

application_record = pd.read_csv('application_record.csv')
credit_record = pd.read_csv('credit_record.csv')

print(application_record.dtypes)
print(credit_record.dtypes)

print(application_record.isnull().sum())
print(credit_record.isnull().sum())

df = pd.merge(application_record, credit_record, on='ID')

df['mau'] = df['STATUS'].apply(lambda x: 1 if x in ['1', '2', '3', '4', '5'] else 0)

print(df['mau'].value_counts(normalize=True))


ID                       int64
CODE_GENDER             object
FLAG_OWN_CAR            object
FLAG_OWN_REALTY         object
CNT_CHILDREN             int64
AMT_INCOME_TOTAL       float64
NAME_INCOME_TYPE        object
NAME_EDUCATION_TYPE     object
NAME_FAMILY_STATUS      object
NAME_HOUSING_TYPE       object
DAYS_BIRTH               int64
DAYS_EMPLOYED            int64
FLAG_MOBIL               int64
FLAG_WORK_PHONE          int64
FLAG_PHONE               int64
FLAG_EMAIL               int64
OCCUPATION_TYPE         object
CNT_FAM_MEMBERS        float64
dtype: object
ID                 int64
MONTHS_BALANCE     int64
STATUS            object
dtype: object
ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPL

#### 2) Vamos montar um metadados

2.1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2.2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável. 
    Dica: 
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis, 
        3. conte o número de categorias dessa variável 
        4. acumule essa informação de 3. na lista que você criou em 1. 
        5. No final, essa lista pode ser atribuída à nossa variável.
2.3. Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}.

In [14]:
metadata = pd.DataFrame(df.dtypes, columns=['tipo_dados'])

qtd_categorias = []
for col in df.columns:
    qtd_categorias.append(df[col].nunique())

metadata['qtd_categorias'] = qtd_categorias

categorical_columns = metadata.loc[(metadata['tipo_dados'] == 'object') & (metadata['qtd_categorias'] > 2)].index
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)


#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [15]:
df_sklearn = df.select_dtypes(exclude='object')

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [16]:
ct = pd.crosstab(df['FLAG_EMAIL'], df['mau'], normalize='index')
ct2 = pd.crosstab(df['FLAG_OWN_CAR'], df['mau'], normalize='index')
print(ct)
print(ct2)

mau                0         1
FLAG_EMAIL                    
0           0.985182  0.014818
1           0.984473  0.015527
mau                  0         1
FLAG_OWN_CAR                    
N             0.984781  0.015219
Y             0.985639  0.014361


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [17]:
df_sklearn.to_csv('base_preparada.csv', index=False)
